In [1]:
import numpy as np
import pandas as pd
import torch

from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
# from tqdm import tqdm
import os 

import pdb
import itertools

from trainer import *
from utils import *
from dfs import *
from pvr_datasets import *

/home/talaei/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# dataset_name = "pointer_majority_3_3"
# max_iter = 2
# verbose_level=1
# run_name = "rbf_var_ablation"
# directory = f"./results/pvr/{dataset_name}/"
# if not os.path.exists(directory):
#         os.makedirs(directory) 
# file_path = directory + run_name
# if not os.path.exists(file_path):
#     torch.save({}, file_path)
# logs = torch.load(file_path)
# setup = {"max iterations": max_iter,
#  "file_path": file_path,
#  "kernel 1": [ ("linear", {}), ("rbf", {'var':1}), ("rbf", {'var':5}), ("rbf", {'var':10}), ("rbf", {'var':20}), ("rbf", {'var':30}), ("rbf", {'var':40})], #, ("rbf", {'var':5}), ("rbf", {'var':10}), ("rbf", {'var':20}), ("rbf", {'var':30}), ("rbf", {'var':40})
#  "kernel 2": [ ("linear", {}), ("rbf", {'var':1}), ("rbf", {'var':5}), ("rbf", {'var':10}), ("rbf", {'var':20}), ("rbf", {'var':30}), ("rbf", {'var':40})],
#  "kernel 3": ["linear", "rbf"],
#  "kernel 4": ["linear", "rbf"],
#  "kernel 5": ["linear", "rbf"],
#  "g 2": ["normalize"],
#  "g 3": ["normalize"],
#  "g 4": ["normalize"],
#  "mixing 2": [[0.75, 0.25], [0.50, 0.50], [0.25, 0.75], [0.00, 1.00]], 
#  "mixing 3": [[0.34, 0.33, 0.33]], 
#  "mixing 4": [[0.25, 0.25, 0.25, 0.25],], 
#  "log_regs 1": [ -3, -2, -1.5, -1, 0, 1],  
#  "log_regs 2": [ -3, -2, -1.5, -1, 0, 1],
#  "log_regs 3": [ -3, -1.5, -1, 0, 1],
#  "log_regs 4": [ -3, -1.5, -1, 0, 1],
#  }
# datasets = load_dataset(dataset_name)
# IKM = IterativeKernelModel(dataset_name, datasets=datasets)
# perform_iteration(IKM, 1, "", verbose_level=verbose_level)
# title = f"pvr/{dataset_name}/{run_name}"
# update_spread_sheet(title, file_path)
        

In [20]:
d, n = 10, 5000
f_name = "f1"
f = f1
dataset_name = f"{f_name}_n={n}_d={d}"
datasets = load_dataset(dataset_name, shuffle=False)
X_test_all = np.array(list(itertools.product([-1.0, 1.0], repeat=d)), dtype=np.float32)
Y_test_all = f(X_test_all).reshape(-1, 1)

In [21]:
(X_train, Y_train, X_test, Y_test) = datasets
datasets = (X_train, Y_train, X_test_all, Y_test_all)
IKM = IterativeKernelModel(dataset_name, datasets=datasets)

Train samples: 4000, Test samples: 1024


In [34]:
kernel = "rbf"
IKM.set_Ds()
IKM.make_kernel_matrices(0, kernel)
IKM.K += 1
IKM.KT += 1
reg = 1
yhat, preds, res, Theta = IKM.solve(reg, verbose=True, max_iter=100)

Exit code: 100
iteration took 0.028516 seconds
Training Error is 0.602879
Test Error is 0.612635
Training Accuracy is 0.793000
Test Accuracy is 0.779297


In [35]:
get_boolean_representation(X_test_all, preds, 2)

{(): 0.009428098797798157,
 (0,): -0.022552892565727234,
 (1,): -0.012314215302467346,
 (2,): 0.24491918087005615,
 (3,): 0.21303090453147888,
 (4,): 0.2082771509885788,
 (5,): 0.23818755149841309,
 (6,): 0.2194097936153412,
 (7,): 0.24616138637065887,
 (8,): 0.00022019445896148682,
 (9,): -0.018865585327148438,
 (0, 1): -0.00015912950038909912,
 (0, 2): -0.005941182374954224,
 (0, 3): -0.006570786237716675,
 (0, 4): -0.005649164319038391,
 (0, 5): 0.006131172180175781,
 (0, 6): 0.005483269691467285,
 (0, 7): 0.006012633442878723,
 (0, 8): 0.0004130154848098755,
 (0, 9): -0.0002738833427429199,
 (1, 2): 0.00042691826820373535,
 (1, 3): -5.0902366638183594e-05,
 (1, 4): -0.00045959651470184326,
 (1, 5): 1.761317253112793e-05,
 (1, 6): 0.00017392635345458984,
 (1, 7): -1.5661120414733887e-05,
 (1, 8): 0.0001484900712966919,
 (1, 9): -7.209181785583496e-05,
 (2, 3): -0.00016151368618011475,
 (2, 4): -0.00032407045364379883,
 (2, 5): -0.000138208270072937,
 (2, 6): 0.0002795606851577759,
 

In [36]:
IKM.set_Ds(yhat=yhat, preds=preds)
IKM.make_kernel_matrices(1, 'rbf')
IKM.combine_kernels([1, 1])

In [37]:
yhat_2, preds_2, res_2, Theta_2 = IKM.solve(1, verbose=True)

Exit code: 400
iteration took 0.088375 seconds
Training Error is 0.574689
Test Error is 0.585782
Training Accuracy is 0.826750
Test Accuracy is 0.813477


In [38]:
get_boolean_representation(X_test_all, preds_2, 3)

{(): 0.010970080271363258,
 (0,): -0.022914251312613487,
 (1,): -0.013371625915169716,
 (2,): 0.2753758430480957,
 (3,): 0.24186153709888458,
 (4,): 0.23510469496250153,
 (5,): 0.268293559551239,
 (6,): 0.24837610125541687,
 (7,): 0.27689725160598755,
 (8,): 0.0008819345384836197,
 (9,): -0.021313900128006935,
 (0, 1): -0.0004926864057779312,
 (0, 2): -0.011545946821570396,
 (0, 3): -0.013448527082800865,
 (0, 4): -0.01136230118572712,
 (0, 5): 0.017124684527516365,
 (0, 6): 0.015495778992772102,
 (0, 7): 0.017002271488308907,
 (0, 8): 0.0007255394011735916,
 (0, 9): -0.0010064635425806046,
 (1, 2): 0.0009522866457700729,
 (1, 3): -0.00019815005362033844,
 (1, 4): -0.0012091752141714096,
 (1, 5): 0.00019773654639720917,
 (1, 6): 0.0006013792008161545,
 (1, 7): 0.00017483346164226532,
 (1, 8): 0.00016394071280956268,
 (1, 9): -0.0002519693225622177,
 (2, 3): 0.004836590960621834,
 (2, 4): 0.004585837945342064,
 (2, 5): -0.00041817687451839447,
 (2, 6): -0.0002812650054693222,
 (2, 7): -

In [69]:
IKM.set_Ds(yhat=yhat_2, preds=preds_2)
IKM.make_kernel_matrices(2, 'rbf')
IKM.combine_kernels([1, 1, 1])

In [70]:
yhat_3, preds_3, res_3, Theta_3 = IKM.solve(1, verbose=True)

Exit code: 400
iteration took 0.080380 seconds
Training Error is 0.364694
Test Error is 0.376065
Training Accuracy is 0.907500
Test Accuracy is 0.907227


In [72]:
get_boolean_representation(X_test_all, preds_3, 3)

{(): 0.011407924816012383,
 (0,): -0.011583068408071995,
 (1,): -0.008931917138397694,
 (2,): 0.27283531427383423,
 (3,): 0.24640776216983795,
 (4,): 0.2387535721063614,
 (5,): 0.26493939757347107,
 (6,): 0.24930578470230103,
 (7,): 0.2711418569087982,
 (8,): 0.0026847217231988907,
 (9,): -0.012538125738501549,
 (0, 1): -0.002525300718843937,
 (0, 2): -0.09210038185119629,
 (0, 3): -0.10329481959342957,
 (0, 4): -0.08864213526248932,
 (0, 5): 0.10826198756694794,
 (0, 6): 0.09759469330310822,
 (0, 7): 0.1069629043340683,
 (0, 8): 0.006438451819121838,
 (0, 9): -0.005649336613714695,
 (1, 2): 0.005056362599134445,
 (1, 3): -0.0021822145208716393,
 (1, 4): -0.009517323225736618,
 (1, 5): -0.0011932579800486565,
 (1, 6): 0.0016563758254051208,
 (1, 7): -0.001428687945008278,
 (1, 8): 0.0017278250306844711,
 (1, 9): -0.0012631900608539581,
 (2, 3): 0.00872022844851017,
 (2, 4): 0.008151320740580559,
 (2, 5): -0.003935407847166061,
 (2, 6): 0.00158026535063982,
 (2, 7): -0.00222853571176528

In [73]:
IKM.set_Ds(yhat=yhat_3, preds=preds_3)
IKM.make_kernel_matrices(3, 'rbf')
IKM.combine_kernels([1, 1, 1, 1])

In [74]:
yhat_4, preds_4, res_4, Theta_4 = IKM.solve(1, verbose=True)

Exit code: 400
iteration took 0.081655 seconds
Training Error is 0.213287
Test Error is 0.219943
Training Accuracy is 1.000000
Test Accuracy is 1.000000


In [75]:
get_boolean_representation(X_test_all, preds_4, 4)

{(): 0.012420343235135078,
 (0,): 0.0024736840277910233,
 (1,): -0.0011693916749209166,
 (2,): 0.2647169530391693,
 (3,): 0.24427296221256256,
 (4,): 0.24034953117370605,
 (5,): 0.25586819648742676,
 (6,): 0.2441941648721695,
 (7,): 0.25630414485931396,
 (8,): 0.003125714138150215,
 (9,): 0.005621140822768211,
 (0, 1): -0.005470169708132744,
 (0, 2): -0.24214035272598267,
 (0, 3): -0.26790979504585266,
 (0, 4): -0.2311897575855255,
 (0, 5): 0.25834253430366516,
 (0, 6): 0.23269079625606537,
 (0, 7): 0.25464263558387756,
 (0, 8): 0.016756946220993996,
 (0, 9): -0.012392831966280937,
 (1, 2): 0.01278902031481266,
 (1, 3): -0.004783155396580696,
 (1, 4): -0.023094383999705315,
 (1, 5): -0.004134336486458778,
 (1, 6): 0.0030178476590663195,
 (1, 7): -0.004973557777702808,
 (1, 8): 0.004962054081261158,
 (1, 9): -0.002694058930501342,
 (2, 3): -0.008092416450381279,
 (2, 4): -0.010089607909321785,
 (2, 5): -0.009184170514345169,
 (2, 6): 0.00987553782761097,
 (2, 7): -0.0012306440621614456,

In [76]:
IKM.set_Ds(yhat=yhat_4, preds=preds_4)
IKM.make_kernel_matrices(4, 'rbf')
IKM.combine_kernels([1, 1, 1, 1, 1])

In [81]:
yhat_5, preds_5, res_5, Theta_5 = IKM.solve(1, verbose=True)

Exit code: 400
iteration took 0.083380 seconds
Training Error is 0.203143
Test Error is 0.210467
Training Accuracy is 1.000000
Test Accuracy is 1.000000


In [82]:
get_boolean_representation(X_test_all, preds_5, 4)

{(): 0.012938320636749268,
 (0,): -0.0007907543331384659,
 (1,): -0.0004961257800459862,
 (2,): 0.26476535201072693,
 (3,): 0.24355080723762512,
 (4,): 0.24095752835273743,
 (5,): 0.25617268681526184,
 (6,): 0.24424302577972412,
 (7,): 0.25616955757141113,
 (8,): 0.0022942572832107544,
 (9,): 0.006611913442611694,
 (0, 1): -0.00507546029984951,
 (0, 2): -0.2464631199836731,
 (0, 3): -0.2720278799533844,
 (0, 4): -0.23480989038944244,
 (0, 5): 0.25987112522125244,
 (0, 6): 0.23454523086547852,
 (0, 7): 0.2563437223434448,
 (0, 8): 0.016517261043190956,
 (0, 9): -0.011495174840092659,
 (1, 2): 0.013851765543222427,
 (1, 3): -0.003972545266151428,
 (1, 4): -0.02245558798313141,
 (1, 5): -0.004346638452261686,
 (1, 6): 0.003279480617493391,
 (1, 7): -0.00517997145652771,
 (1, 8): 0.005006525665521622,
 (1, 9): -0.0025009552482515574,
 (2, 3): -0.01363193616271019,
 (2, 4): -0.015811851248145103,
 (2, 5): -0.00898807868361473,
 (2, 6): 0.010587547905743122,
 (2, 7): -0.0007494017481803894,
